In [1]:
import numpy as np
import pandas as pd
import emojis
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [38]:
data=pd.read_csv("train.csv")
ok=pd.read_csv("Test.csv")
test=[]
for j in range (0,len(ok["TEXT"])):
    test.append(ok["TEXT"][j])
mapping=pd.read_csv("mapping.csv")
mapping=mapping.to_dict()["emoticons"]
data=data.drop('Unnamed: 0', axis=1)

😜


In [3]:
X=data["TEXT"].values
Y=data["Label"].values

In [4]:
file=open("glove.twitter.27B.100d.txt","r",encoding="utf8")
content=file.readlines()
file.close()

In [5]:
embeddings={}
for line in content:
    line=line.split()
    embeddings[line[0]]=np.array(line[1:],dtype=float)


In [6]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

In [7]:
def padding(data):
    L=[]
    for j in range (0,len(data)):
        L.append(len(data[j]))
    n=max(L)
    for j in range (0,len(data)):
        for k in range (0,n-L[j]):
            data[j].append(0)
    return data,n

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index
Xtokens = tokenizer.texts_to_sequences(X)
maxlen=get_maxlen(Xtokens)
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')

In [9]:
Ytrain=to_categorical(Y)

In [10]:
# def intialize_emb_matrix(file):
#     embedding_matrix = []
#     for line in file:
#         values = line.split()
#         word = values[0]
#         embedding = np.array(values[1:], dtype='float64')
#         embedding_matrix[word] = embedding

#     return embedding_matrix 
# embedding_matrix=intialize_emb_matrix(content)

In [11]:
embed_size = 100
embedding_matrix = np.zeros((len(word2index)+1, embed_size))
for word, i in word2index.items():
    if word in embeddings:
        embed_vector = embeddings[word]
        embedding_matrix[i] = embed_vector

In [20]:
# model = Sequential([
#     Embedding(input_dim = len(word2index) + 1,
#               output_dim = embed_size,
#               input_length = maxlen,
#               weights = [embedding_matrix],
#               trainable = False
#              ),
    
#     LSTM(units = 256, return_sequences = True),
#     LSTM(units = 128),
#     Dense(5, activation = 'softmax')
# ])


model=Sequential([
    Embedding(input_dim = len(word2index)+1,
              output_dim = 100,
              input_length = maxlen,
              weights = [embedding_matrix],
              trainable = False
             ),
])

model.add(LSTM(units = 256, return_sequences=True)
model.add(LSTM(units=128))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=20, activation='softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [21]:
# res = model.fit(Xtrain, Ytrain, validation_split=0.2, batch_size=32, epochs=10, verbose=2)

In [51]:
test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')
y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

812/812 [==============================] - 34s 39ms/step


In [54]:
ans=[]
for i in range(len(test)):
    ans.append(mapping[y_pred[i]])
df=pd.DataFrame({"Text":test,"Survived":ans})
df.to_csv("submission_twitter_emoji_predictor.csv",index=False)